In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os
import random as rng
from tqdm import tqdm

import seaborn as sns


%matplotlib inline

rng.seed(12345)

In [2]:
# path
figure_log_root = Path('/cluster/scratch/jingyli/interpretable/visualization')

In [3]:
def read_watershed_results(DATE, TIME, CHANNEL, W, MODEL):
    filename = f"{DATE}_{TIME}_C{CHANNEL}-W{W}.npy"
    filepath = figure_log_root / f"{MODEL}" / f"{DATE}_{TIME}" / "watershed"/ filename
    if os.path.exists(filepath):
        arr = np.load(filepath)
        return arr
    else:
        return None

## Get watershed results for a target window in different time/days

In [4]:
MODEL = "resUnet"
CHANNEL = 0

W = f"{18}_{6}"
DATES = ['2019-07-01',]
#                  '2019-07-11',
#                  '2019-07-21',
#                  '2019-07-31',]
TIMES = np.arange(0,276,12)

# To store multiple watershed marker layers
watershed_layer = {}


for d in tqdm(DATES):
    for t in TIMES:
        arr = read_watershed_results(d, t, CHANNEL, W, MODEL)
        if arr is None:
            continue
        markers = arr[...,0]
        markers = np.where((markers > 0) & (markers < 255), 1, 0)
        
        watershed_layer[f"{d}_{t}"] = markers

print(f"Loaded {len(watershed_layer)} in total")

100%|██████████| 1/1 [00:00<00:00, 196.84it/s]

Loaded 0 in total


In [10]:
watershed_layer_arr = np.array([v for k, v in watershed_layer.items()])
watershed_layer_names = [k for k, v in watershed_layer.items()]

{'2019-07-01_144': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])}

In [ ]:
## Intersection between the two
def calculate_iou(layer1, layer2):

    arr = np.argwhere(layer1==1)
    arr2 = np.argwhere(layer2==1)

    intersection = list(set(arr) & set(arr2))
    union = list(set(arr) | set(arr2))

    iou = len(intersection)/len(union)
    
    return iou

In [ ]:
iou_arr = np.zeros((watershed_layer_arr.shape[0], watershed_layer_arr.shape[0]))
for i in range(watershed_layer_arr.shape[0]):
    for j in range(watershed_layer_arr.shape[0]):
        iou_arr[i, j] = calculate_iou(watershed_layer_arr[i], watershed_layer_arr[j])

In [ ]:
from matplotlib.colors import LogNorm
sns.set(rc={'figure.figsize':(15,12)})
ax = sns.heatmap(iou_arr, annot=True, fmt="d", linewidths=.5, square=True, norm=LogNorm(), cmap="YlGn")
plt.savefig('temporal_test.png', bbox_inches="tight")
plt.show()